# AgentGuard Quickstart

**Stop runaway AI agents before they drain your budget.**

This notebook demonstrates AgentGuard's BudgetGuard — a runtime guard that kills agents when they exceed a dollar limit. No API keys needed.

[![GitHub](https://img.shields.io/github/stars/bmdhodl/agent47?style=social)](https://github.com/bmdhodl/agent47)

In [ ]:
!pip install -q agentguard47

## 1. Set a budget and simulate agent calls

BudgetGuard tracks tokens, API calls, and dollar cost. When any limit is hit, it raises `BudgetExceeded` and the agent stops immediately.

In [ ]:
from agentguard import BudgetGuard, BudgetExceeded

budget = BudgetGuard(
    max_cost_usd=1.00,
    warn_at_pct=0.8,
    on_warning=lambda msg: print(f"WARNING: {msg}")
)

print("Simulating agent making LLM calls with a $1.00 budget...\n")
for i in range(1, 20):
    try:
        budget.consume(tokens=500, calls=1, cost_usd=0.12)
        print(f"  Call {i}: ${budget.state.cost_used:.2f} spent")
    except BudgetExceeded as e:
        print(f"\n  STOPPED at call {i}: {e}")
        print(f"  Total: ${budget.state.cost_used:.2f} | {budget.state.calls_used} calls")
        break

## 2. Loop detection

LoopGuard detects when an agent repeats the same tool call — a common failure mode where agents get stuck in infinite loops.

In [ ]:
from agentguard import LoopGuard, LoopDetected

loop_guard = LoopGuard(max_repeats=3, window=6)

print("Simulating an agent stuck in a loop...\n")
for i in range(1, 10):
    try:
        loop_guard.check("search_docs")  # same tool every time
        print(f"  Call {i}: search_docs (ok)")
    except LoopDetected as e:
        print(f"\n  LOOP DETECTED at call {i}: {e}")
        break

## 3. Full tracing with guards

Combine tracing and guards for complete observability. Traces output as JSONL — no dashboard required.

In [ ]:
from agentguard import Tracer, BudgetGuard, LoopGuard, JsonlFileSink

tracer = Tracer(
    sink=JsonlFileSink("demo_traces.jsonl"),
    service="my-agent",
    guards=[
        BudgetGuard(max_cost_usd=5.00),
        LoopGuard(max_repeats=3),
    ],
)

with tracer.trace("agent.run") as span:
    span.event("reasoning", data={"thought": "search docs"})
    with span.span("tool.search", data={"query": "quantum computing"}):
        pass  # your tool logic here

print("Trace written to demo_traces.jsonl")
print("Run `agentguard report demo_traces.jsonl` to see a summary.")

## Next steps

- [Full documentation](https://github.com/bmdhodl/agent47)
- [LangChain integration](https://github.com/bmdhodl/agent47#langchain)
- [Auto-instrument OpenAI/Anthropic](https://github.com/bmdhodl/agent47#openai--anthropic-auto-instrumentation)